In [58]:
#Player stats
# Get all player stats for each year
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog, playercareerstats, playercareerstats, playergamelogs
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelogs, leaguegamelog, boxscoreadvancedv2


import numpy as np
import pandas as pd
import os
import time


13    1.8
Name: NET_RATING, dtype: float64


In [71]:

def get_team_for_year(start_year):
    season = '{}-{}'.format(start_year, str(start_year + 1)[-2:])
    year_frame = leaguegamelog.LeagueGameLog(season=season).get_data_frames()
    
    return year_frame

frame = pd.DataFrame(get_team_for_year(2012)[0])
print(frame['TEAM_NAME'].unique())

['Los Angeles Lakers' 'Dallas Mavericks' 'Cleveland Cavaliers'
 'Boston Celtics' 'Miami Heat' 'Washington Wizards' 'Houston Rockets'
 'Los Angeles Clippers' 'Memphis Grizzlies' 'Toronto Raptors'
 'Indiana Pacers' 'Utah Jazz' 'Portland Trail Blazers' 'Phoenix Suns'
 'Golden State Warriors' 'Sacramento Kings' 'Chicago Bulls'
 'Philadelphia 76ers' 'Denver Nuggets' 'New Orleans Hornets'
 'San Antonio Spurs' 'Detroit Pistons' 'Oklahoma City Thunder'
 'Minnesota Timberwolves' 'Milwaukee Bucks' 'Orlando Magic'
 'New York Knicks' 'Charlotte Bobcats' 'Atlanta Hawks' 'Brooklyn Nets']


In [29]:
# TODO Find efficient weight paramater (ELO) to be used in linear regression
# TODO plot data, linear regression may not capture much data due to distribition
# TODO plot player and game data with various plots (list of plots on note)


In [61]:
players = pd.DataFrame(pd.read_csv("players.csv"))




# start with this function to find all players played in a seaosn. This will return a column of unique player IDs that played for a season
def get_players_for_season(season_start):
    file_location = "../player_season/player_season_data/season_year_start_player_data_" + str(season_start)
    season = pd.read_csv(file_location)
    players_id = season['PLAYER_ID'].drop_duplicates()
    
    return players_id


#use this to find a players ID. not needed but for convienience
def find_player_id(player_full_name):
    for i in range(0, len(players)):
        player = players.iloc[i]
        if player["full_name"] == player_full_name:
            return player['id']
        
# once you have that list of player ID use this function next and just input the season start you want, and keep calling this function and it will return the players stat
def get_player_stat_for_season(season_start, player_name = None):
    # check if player played for that season.
    # if not played return program with error
    
    file_location = "../player_season/player_season_data/season_year_start_player_data_" + str(season_start)
    season = pd.read_csv(file_location)
    return(season)



def get_stat_for_player(season, player_id):
     player_stat = season[season['PLAYER_ID'] == player_id]        
     return player_stat
             
# data frame object
# Lebron test

# this line of code shows you how to get lebrosn stat for the 2014 seaosn start


# player_season_stat = get_stat_for_player(get_player_stat_for_season(season_start="2014"),player_id=find_player_id("LeBron James"))

# use this function to get all the data required for each player including box scores
def total_function_for_players_stat_in_season(season_start_year):
    

    players = get_players_for_season(season_start_year)



    path = './player_data_for_' + str(season_start_year)
    box_score_path = './box_score_for_players_in_season' + str(season_start_year)
    box_score_each_player_season = './box_score_for_each_player_in_season' + str(season_start_year)
    final_box_data_player_data = './final_box&data_player_data_for_' + str(season_start_year)
    

    # remove try block to prevent duplicates after
    try: 
        os.mkdir(path)
    except FileExistsError:
        pass
   

    # find all games in a season then create a csv file for the boxscores
    try: 
        os.mkdir(box_score_path)
    except FileExistsError:
        pass

    player_season_stat = get_player_stat_for_season(season_start=season_start_year)

    for player in player_season_stat['PLAYER_ID'].unique():
        stat = get_stat_for_player(player_season_stat, player_id=player)
        stat['NET_RANKING'] = np.NAN
        path_player = path + '/' + str(player) + '.csv'
        stat.to_csv(path_player)

    count = 0
    # very slow
    for game_id in player_season_stat['GAME_ID'].unique():
        game_id = '00' + str(game_id)
        box_score_score_for_game = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id).get_data_frames()[0]
        time.sleep(0.6)
        
        print(count)
        count = count + 1
        box_score_file_path = box_score_path + '/' + game_id + '.csv'
        box_score_score_for_game.to_csv(box_score_file_path)
    

    # gather for each player

    try: 
        os.mkdir(box_score_each_player_season)
    except FileExistsError:
        pass
    # now go through each box score file and then as you iterate through each row add
    # them into respective player box score file

    for game_file in os.listdir(box_score_path):
        if not game_file.startswith('.'):
        
            box_score_for_game_file = pd.read_csv(box_score_path + '/' + str(game_file))

            for id in box_score_for_game_file['PLAYER_ID']:
                try:
                    file = pd.read_csv(box_score_each_player_season + '/' + str(id) + '.csv')
                    entry = box_score_for_game_file.loc[box_score_for_game_file['PLAYER_ID'] == id]
                    concat_df = pd.concat([file,entry],ignore_index=True, join='outer')
                    concat_df.to_csv(box_score_each_player_season + '/' + str(id) + '.csv',index=False)


                except FileNotFoundError:
                    empty_frame = pd.DataFrame(columns=['GAME_ID','TEAM_ID','TEAM_ABBREVIATION',
                                                                'TEAM_CITY','PLAYER_ID','PLAYER_NAME','NICKNAME','START_POSITION','COMMENT','MIN',
                                                                'E_OFF_RATING','OFF_RATING','E_DEF_RATING','DEF_RATING','E_NET_RATING','NET_RATING','AST_PCT',
                                                                'AST_TOV','AST_RATIO','OREB_PCT','DREB_PCT','REB_PCT','TM_TOV_PCT','EFG_PCT','TS_PCT',
                                                                'USG_PCT','E_USG_PCT','E_PACE','PACE','PACE_PER40','POSS','PIE'])
                
                    entry = box_score_for_game_file.loc[box_score_for_game_file['PLAYER_ID'] == id]
                    concat_df = pd.concat([empty_frame,entry],ignore_index=True,join='outer')
                    concat_df.to_csv(box_score_each_player_season + '/' + str(id) + '.csv',index=False)
 
    # get the list of files then iterate through that list

    try: 
        os.mkdir(final_box_data_player_data)
    except FileExistsError:
        pass
    
    for  player_data_file_d in os.listdir(path):
        if not player_data_file_d.startswith('.'):

            each_player_data = pd.read_csv(path + '/' + player_data_file_d)
            
            each_player_box = pd.read_csv(box_score_each_player_season + '/' + player_data_file_d)
            
            merged = each_player_data.join(each_player_box.set_index('GAME_ID'), on='GAME_ID',lsuffix='_left')
            
            merged.to_csv(final_box_data_player_data+'/' + player_data_file_d)
        
        

    
# might take an hour or two, just to download the box scores might take a while 
total_function_for_players_stat_in_season('2005')
total_function_for_players_stat_in_season('2006')
total_function_for_players_stat_in_season('2007')
total_function_for_players_stat_in_season('2008')
total_function_for_players_stat_in_season('2009')       
total_function_for_players_stat_in_season('2010')
total_function_for_players_stat_in_season('2011')
total_function_for_players_stat_in_season('2012')
total_function_for_players_stat_in_season('2013')
total_function_for_players_stat_in_season('2014')
total_function_for_players_stat_in_season('2015')
total_function_for_players_stat_in_season('2016')
total_function_for_players_stat_in_season('2017')
total_function_for_players_stat_in_season('2018')
total_function_for_players_stat_in_season('2019')
total_function_for_players_stat_in_season('2020')
total_function_for_players_stat_in_season('2021')
total_function_for_players_stat_in_season('2022')









/var/folders/jm/cm59457511v80mtdqy5_hx7c0000gn/T/ipykernel_84255/2552078903.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat['NET_RANKING'] = np.NAN
/var/folders/jm/cm59457511v80mtdqy5_hx7c0000gn/T/ipykernel_84255/2552078903.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat['NET_RANKING'] = np.NAN
/var/folders/jm/cm59457511v80mtdqy5_hx7c0000gn/T/ipykernel_84255/2552078903.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
# data preprocessing and cleaning and analysis

# for every player_stat_object 

# add column for team versed
# go through every row and after very 'vs' find the teamId and column (against team column)
# might not need cause data is already connected through gameID

# TODO Dealing with N/A values with old data
# TODO probably better of just deleting rows with N/A values 
# TODO maube estimate


# read from a player csv file or from a function
player_season_stat = pd.read_csv("lebron_test_file.csv")

player_season_stat = player_season_stat.drop('TEAM_NAME', axis=1)
player_season_stat = player_season_stat.drop('PLAYER_NAME', axis=1)
player_season_stat = player_season_stat.drop('NICKNAME', axis=1)
player_season_stat = player_season_stat.drop('TEAM_ABBREVIATION', axis=1)
player_season_stat = player_season_stat.drop('WNBA_FANTASY_PTS_RANK', axis=1)
player_season_stat = player_season_stat.drop('NBA_FANTASY_PTS_RANK', axis=1)
player_season_stat = player_season_stat.drop('NBA_FANTASY_PTS', axis=1)
player_season_stat = player_season_stat.drop('WNBA_FANTASY_PTS', axis=1)
# Maybe keep season year
player_season_stat = player_season_stat.drop('SEASON_YEAR', axis=1)
player_season_stat = player_season_stat.drop('Unnamed: 0.1', axis=1)
player_season_stat = player_season_stat.drop('Unnamed: 0', axis=1)
player_season_stat['GAME_DATE'] = pd.to_datetime(player_season_stat['GAME_DATE'])


# drop MIN_SEC since it is an object 
player_season_stat = player_season_stat.drop('MIN_SEC', axis=1)




def home_away_map(x):
    try:
        index = x.index('@')
        return 0
    except ValueError:
        return 1


from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
player_season_stat['WL'] = lb.fit_transform(player_season_stat['WL']) # check type, maybe convert to boolean or osemthing
player_season_stat['MATCHUP'] =  player_season_stat['MATCHUP'].map(home_away_map)
print(player_season_stat.info())
player_season_stat.to_csv('lebron_proccessed_test_file.csv')




# choose columsn to standardise.


# move Win lose column to first column.
# plot adn see 










In [ ]:
# PYTORCH
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

my_matrix = player_season_stat.corr(method="spearman").round(2)

#my_matrix.style.background_gradient(cmap="Greens")

#pd.plotting.scatter_matrix(my_matrix, figsize=(8,8))

my_mask = np.triu(np.ones_like(my_matrix,dtype=bool))

plt.figure(figsize=(100,80)) #width, height
sns.heatmap(my_matrix, cmap="Blues", vmin=0, vmax=1,
           annot=True, fmt="0.2f", square=True, mask=my_mask)

plt.savefig('./data_plots/lebron_corr_matrix_test.png')





